In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

**READING THE DATA** 

In [ ]:
train = pd.read_csv("train.csv", index_col = 'Id')
print(train.shape)
train.head()

In [ ]:
test = pd.read_csv("test.csv", index_col = 'Id')
print(test.shape)
test.head()

In [ ]:
submission = pd.read_csv("submission.csv", index_col = 'Id')
print(submission.shape)
submission.head()

In [ ]:
train.info()

In [ ]:
train.describe().transpose()

In [ ]:
y = train['targets']
train_cols = [col for col in train.columns if col not in ['targets']]
X = train[train_cols]

**SPLITTING THE DATA**

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=True)

**MODEL**

- Building the model

In [ ]:
def build_model():
    model = tf.keras.Sequential([
                               tf.keras.layers.Dense(4,kernel_initializer='normal', activation = 'relu', input_shape = (16,)), 
                               tf.keras.layers.Dense(4,kernel_initializer='normal',activation = 'relu'),
                               tf.keras.layers.Dense(1,kernel_initializer='normal', activation = 'linear')])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    los = tf.keras.losses.MeanSquaredError()
    metric = tf.keras.metrics.MeanSquaredError()
    model.compile(loss = los, optimizer = optimizer, metrics = metric)

    return model

In [ ]:
model = build_model()
model.summary()

- Training the Model

In [ ]:
class PrintDot (tf.keras.callbacks.Callback) :
    def on_epoch_end(self, epoch, logs) :
        if epoch % 100 == 0 :
            print('')
        print('.', end = '')

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_mean_squared_error', patience = 10)
history = model.fit(X_train, y_train, epochs=1000, validation_split = 0.2, verbose =0, callbacks = [early_stop, PrintDot()])

In [ ]:
plt.title("Mean squared error")
plt.plot(history.history['mean_squared_error'])
plt.plot(history.history['val_mean_squared_error'])
plt.show()

In [ ]:
# training predictions
tpred = model.predict(X_train)
tscore = mean_squared_error(y_train, tpred)

# validation predictions
vpred = model.predict(X_test)
vscore = mean_squared_error(y_test, vpred)

print(f'Training score   - {tscore:.3f} \nValidation score - {vscore:.3f}')

In [ ]:
# get the data and transform
final_test = test.values
final_test = scaler.transform(final_test)

# predict
preds = model.predict(final_test)

In [ ]:
# put it in submission file
submission['targets'] = preds

# save the file
submission.to_csv("submission.csv")